import ipotrant libraries

In [1]:
import ROOT
import math
from array import array
import numpy as np
#from ROOT import gROOT, AddressOf
import multiprocess as mp
%jsroot on

Welcome to JupyROOT 6.29/01


Set common variables

In [2]:
files_path = "/home/yoren/bnl/PHENIX/Trees/"
file_names = ["Tree_orig","Tree_0","Tree_1","Tree_2","Tree_3","Trees_6M","Trees_6M","Trees_6M","Trees_6M"]
treenames = ["AnalysisTree","AnalysisTreeEmbed","AnalysisTreeEmbed","AnalysisTreeEmbed","AnalysisTreeEmbed","AnalysisTree","AnalysisTree","AnalysisTree","AnalysisTree"]
outnames = ["HeAu_sim_hists_orig","HeAu_sim_hists_embed_0","HeAu_sim_hists_embed_1","HeAu_sim_hists_embed_2","HeAu_sim_hists_embed_3"\
            ,"HeAu_sim_hists_6M_0","HeAu_sim_hists_6M_1","HeAu_sim_hists_6M_2","HeAu_sim_hists_6M_3"]
E_def_core_cut = 0.500
Chi2_def_cut = 3
alpha_def_cut = 0.6
distance_def_cut = 8

Option = 8

In [3]:
class photon:
  def __init__(self, i, E, x, y, z, vertex, arm, sec, chi2, id):
    self.i = i
    self.E = E
    self.x = x
    self.y = y
    self.z = z
    self.vertex = vertex
    self.arm = arm
    self.sec = sec
    self.chi2 = chi2
    self.id = id
    self.r = (self.x**2+self.y**2+(self.z-self.vertex)**2)**0.5+0.001
    self.px = self.E * self.x / self.r
    self.py = self.E * self.y / self.r
    self.pz = self.E * (self.z-self.vertex) / self.r
    self.pt = (self.px**2+self.py**2)**0.5

  def __str__(self):
    return f"i={self.i}, px={round(self.px,1)}, py={round(self.py,1)}, pz={round(self.pz,1)}, \
    pt={round(self.pt,1)}, E={round(self.E,1)}, x={round(self.x,1)}, y={round(self.y,1)}, z={round(self.z,1)}, \
      arm={self.arm}, sec={self.sec}, chi2={round(self.chi2,1)}, id={self.id}"

In [4]:
class hadron:
  def __init__(self, i, x, y, z, id):
    self.i = i
    self.x = x
    self.y = y
    self.z = z
    self.id = id

  def __str__(self):
    return f"i={self.i}, x={self.x}, y={self.y}, z={self.z}, id={self.id}"

Define structures

In [5]:
def is_photon_good(photon):
    if photon.E > E_def_core_cut and photon.chi2 < Chi2_def_cut: return True
    return False

In [6]:
def is_hadron_good(hadron):
    if hadron.x<-900 or hadron.y<-900 or hadron.z<-900: return False
    return True

In [7]:
def alpha(photon1, photon2):
    alpha = abs(photon1.E-photon2.E)/(photon1.E+photon2.E)
    return alpha

In [8]:
def Distance(photon1, photon2):
    Distance = ((photon1.x-photon2.x)**2+(photon1.y-photon2.y)**2+(photon1.z-photon2.z)**2)**0.5
    return Distance

In [9]:
def is_pair_good(photon1, photon2):
    if photon1.arm == photon2.arm and abs(photon1.sec-photon2.sec) < 2: 
        if alpha(photon1, photon2) < alpha_def_cut and Distance(photon1, photon2) > distance_def_cut: return True
    return False

In [10]:
def inv_mass(photon1, photon2):
    e1, px1, py1, pz1 = photon1.E, photon1.px, photon1.py, photon1.pz
    e2, px2, py2, pz2 = photon2.E, photon2.px, photon2.py, photon2.pz
    m2 = (e1+e2)**2 - (px1+px2)**2 - (py1+py2)**2 - (pz1+pz2)**2
    if m2<0: return 0
    return math.sqrt(m2)

In [11]:
dphi_mean_fits_name, dphi_sigma_fits_name = "fit_mean_functions_emc_min_r_dphi_0-88%", "fit_functions_sigma_emc_min_r_dphi_0-88%"
dzed_mean_fits_name, dzed_sigma_fits_name = "fit_mean_functions_emc_min_r_dz_0-88%", "fit_functions_sigma_emc_min_r_dz_0-88%"
inFileFuncs = ROOT.TFile.Open("input/HeAu_emcid.root","read")
dphi_mean_fits, dphi_sigma_fits = inFileFuncs.Get(dphi_mean_fits_name), inFileFuncs.Get(dphi_sigma_fits_name)
dzed_mean_fits, dzed_sigma_fits = inFileFuncs.Get(dzed_mean_fits_name), inFileFuncs.Get(dzed_sigma_fits_name)

In [12]:
class real_event:
  
  def __init__(self,N_hadrons):
    self.N_hadrons = N_hadrons
    self.hadron = []
  
  def __str__(self):
    return f"N_hadrons = {self.N_hadrons}"
  
  def add_hadron(self, hadron):
    self.hadron.append(hadron)
  
  def set_N_good_hadrons(self):
    self.N_hadrons = len(self.hadron)

In [13]:
def get_real_events(ii):
    inFileCharged = ROOT.TFile.Open("input/tracks_new.root","read")
    tree_real_event = inFileCharged.Get(f"TracksTree{(ii-4)%5}")
    N_real_ev = tree_real_event.GetEntries()
    my_real_events=[]

    #def get_real_event(tree_real_event, ievent):
    my_real_events=[]
    for ev in tree_real_event:
        N_ch = len(ev.trk_emc_x)
        my_real_event = real_event(N_ch)
        for ihadron in range(N_ch):
            my_real_hadron = hadron(0,ev.trk_emc_x[ihadron],ev.trk_emc_y[ihadron],ev.trk_emc_z[ihadron],ev.trk_emc_id[ihadron])
            if is_hadron_good(my_real_hadron):
                my_real_event.add_hadron(my_real_hadron)
        my_real_event.set_N_good_hadrons()
        my_real_events.append(my_real_event)

    return my_real_events, N_real_ev

In [14]:
def is_charged_vetoed(photon, event):
    r_min = 10
    for hadron in event.hadron:
        dphi = math.atan2(photon.y, photon.x) - math.atan2(hadron.y, hadron.x)
        dzed = photon.z - hadron.z
        pt = photon.pt
        if pt<0.5: pt = 0.5
        if pt>15: pt =15
        sdphi = (dphi - dphi_mean_fits.Eval(pt))/dphi_sigma_fits.Eval(pt)
        sdzed = (dzed - dzed_mean_fits.Eval(pt))/dzed_sigma_fits.Eval(pt)
        r = (sdphi**2+sdzed**2)**0.5
        if r < r_min: r_min = r
    if r_min < 3: return True
    return False

In [15]:
def is_charged_vetoed_sigma(photon, event):

    for hadron in event.hadron:
        dphi = math.atan2(photon.y, photon.x) - math.atan2(hadron.y, hadron.x)
        dzed = photon.z - hadron.z
        sdphi = (dphi - dphi_mean_fits.Eval(photon.pt))/dphi_sigma_fits.Eval(photon.pt)
        sdzed = (dzed - dzed_mean_fits.Eval(photon.pt))/dzed_sigma_fits.Eval(photon.pt)
    
        if abs(sdphi) < 2 or abs(sdzed) < 2 : return True
    return False

In [16]:
class event:
  
  def __init__(self, i, N_photons, N_hadrons, vertex):
    self.i = i
    self.N_photons = N_photons
    self.N_hadrons = N_hadrons
    self.vertex = vertex
    self.N_good_photons = self.N_photons
    self.N_good_hadrons = self.N_hadrons
    self.photon = []
    self.hadron = []
    self.m_inv = []  
  
  def __str__(self):
    return f"i = {self.i}, vertex = {round(self.vertex)}, N_photons = {self.N_photons}, N_hadrons = {self.N_hadrons}, N_good_photons = {self.N_good_photons}, N_good_hadrons = {self.N_good_hadrons}"
  
  def add_hadron(self, hadron):
    self.hadron.append(hadron)

  def add_photon(self, photon):
    self.photon.append(photon)
  
  def set_N_good_photons(self):
    self.N_good_photons = len(self.photon)
  
  def set_N_good_hadrons(self):
    self.N_good_hadrons = len(self.hadron)
  
  def get_inv_masses(self):
    for iphoton1 in range(min(self.N_good_photons,200)):
      for iphoton2 in range(iphoton1+1,min(self.N_good_photons,200)):
        photon1 = self.photon[iphoton1]
        photon2 = self.photon[iphoton2]
        if is_pair_good(photon1,photon2):
          self.m_inv.append([inv_mass(photon1,photon2), photon1.pt + photon2.pt, self.vertex])
    return self.m_inv

Create TTree creator

In [17]:
class pool:
  
  def __init__(self, depth, VTX):
    self.depth = depth
    self.VTX = VTX
    self.N_events = [0]*(len(self.VTX)-1)
    self.photons_in_event = [ [] for _ in range(len(self.VTX)-1)]
    self.m_inv = []  
  
  def __str__(self):
    return f"depth = {self.depth}"
  

  def add_photons(self, event):
    for i in range(len(self.N_events)):
      if(event.vertex> self.VTX[i], event.vertex < self.VTX[i+1]): 
        self.N_events[i]+=1
        self.photons_in_event[i].append(event.photon)

  
  def get_inv_masses(self):
    is_filled = False
    for iev in range(len(self.N_events)):
      if self.N_events[iev]>=self.depth:
        self.m_inv = []
        is_filled = True
        for iphoton1 in range(len(self.photons_in_event[iev][0])):
          for second_event in range(1,self.depth):
            for iphoton2 in range(len(self.photons_in_event[iev][second_event])):
              photon1 = self.photons_in_event[iev][0][iphoton1]
              photon2 = self.photons_in_event[iev][second_event][iphoton2]
              if is_pair_good(photon1,photon2):
                self.m_inv.append([inv_mass(photon1,photon2), photon1.pt + photon2.pt, 0.5*(photon1.vertex + photon2.vertex)])
        self.N_events[iev] = self.depth-1
        self.photons_in_event[iev].pop(0)
    if is_filled: return self.m_inv
    else: return []


Create file reader

In [18]:
def hist_creator(nameFG = "hist_inv_mass_FG", nameBG = "hist_inv_mass_BG", nameP = "hist_pt_orig"):
    hist_inv_mass_FG = ROOT.TH3D(nameFG,nameFG,1000,0,1,40,0,20,60,-30,30)
    hist_inv_mass_FG.SetDirectory(ROOT.nullptr)
    hist_inv_mass_BG = ROOT.TH3D(nameBG,nameBG,1000,0,1,40,0,20,60,-30,30)
    hist_inv_mass_BG.SetDirectory(ROOT.nullptr)
    hist_pt_orig = ROOT.TH2D(nameP,nameP,40,0,20,60,-30,30)
    hist_pt_orig.SetDirectory(ROOT.nullptr)

    return [hist_inv_mass_FG, hist_inv_mass_BG, hist_pt_orig]

In [19]:
def main_func(ii, Ntr, jj):

    [hist_inv_mass_FG, hist_inv_mass_BG, hist_pt_orig] = hist_creator(f"hist_inv_mass_FG{ii}", f"hist_inv_mass_BG{ii}", f"hist_pt_orig{ii}")
    if ii>4: my_real_events, N_real_ev = get_real_events(ii)
    inFile = ROOT.TFile.Open(files_path+file_names[ii]+'.root',"read")
    tree_read_event = inFile.Get(treenames[ii])
    Nev = tree_read_event.GetEntries()
    Low, High = int(Nev/Ntr*jj), int(Nev/Ntr*(jj+1))
    print(Nev)
    
    my_pool = pool(3,[-30,-8,8, 30])
    for ievent in range(Low,High):
        tree_read_event.GetEntry(ievent)
        ev = tree_read_event
        my_event  = event(ievent, ev.n_gammas, ev.n_charged, ev.bbc_vertex)    
        hist_pt_orig.Fill((ev.p_orig[0]**2+ev.p_orig[1]**2)**0.5,ev.bbc_vertex)
        
        for ihadron in range(ev.n_charged):
            my_hadron = hadron(ihadron,ev.trk_emcxs[ihadron],ev.trk_emcys[ihadron],ev.trk_emczs[ihadron],ev.trk_ids[ihadron])
            if is_hadron_good(my_hadron):
                my_event.add_hadron(my_hadron)
        my_event.set_N_good_hadrons()
        
        for iphoton in range(my_event.N_photons):
            my_photon = photon(iphoton, ev.g_es[iphoton], ev.g_xs[iphoton], ev.g_ys[iphoton], ev.g_zs[iphoton], ev.bbc_vertex, ev.g_arms[iphoton], ev.g_secs[iphoton], ev.g_chi2s[iphoton], ev.g_ids[iphoton])
            #my_real_event = get_real_event(tree_real_event,ievent%N_real_ev)
            if is_photon_good(my_photon):
                if ii<5 or (not is_charged_vetoed(my_photon, my_event) and not is_charged_vetoed(my_photon, my_real_events[ievent%N_real_ev])):
                    my_event.add_photon(my_photon)
        my_event.set_N_good_photons()
        if my_event.N_good_photons>0: my_pool.add_photons(my_event)
        
        my_inv_masses = my_event.get_inv_masses()
        my_inv_masses_bg = my_pool.get_inv_masses()
        for my_inv_mass in my_inv_masses:
            hist_inv_mass_FG.Fill(my_inv_mass[0],my_inv_mass[1],my_inv_mass[2])
        for my_inv_mass_bg in my_inv_masses_bg:
            hist_inv_mass_BG.Fill(my_inv_mass_bg[0],my_inv_mass_bg[1],my_inv_mass_bg[2])

    inFile.Close()
    print("all good", Low, High)

    return [hist_inv_mass_FG, hist_inv_mass_BG, hist_pt_orig, High-Low]

In [20]:
Ntr = 4
params = [(Option, Ntr, i) for i in range(Ntr)]
print(params)
pool = mp.Pool(Ntr)
output_array = pool.starmap(main_func, params)
pool.close()

[(8, 4, 0), (8, 4, 1), (8, 4, 2), (8, 4, 3)]
5990000
5990000
5990000
5990000
all good 4492500 5990000
all good 2995000 4492500
all good 1497500 2995000
all good 0 1497500


In [21]:
[hist_inv_mass_FG, hist_inv_mass_BG, hist_pt_orig, Nev] = [output_array[0][0], output_array[0][1], output_array[0][2], output_array[0][3]]
for i in range(1, len(output_array)):
    hist_inv_mass_FG.Add(output_array[i][0])
    hist_inv_mass_BG.Add(output_array[i][1])
    hist_pt_orig.Add(output_array[i][2])
    Nev+=output_array[i][3]

In [22]:
FileOut = ROOT.TFile("input/"+outnames[Option]+".root","recreate")
hist_inv_mass_FG.Write()
hist_inv_mass_BG.Write()
hist_pt_orig.Write()
FileOut.Close()

In [23]:

c1 = ROOT.TCanvas("c1","c1",720,720)
hist = hist_inv_mass_FG.ProjectionX("proj",1,39,1,59)
hist_BG = hist_inv_mass_BG.ProjectionX("proj_BG",1,39,1,59)
hist.Draw()
hist_BG.Scale(hist.Integral(hist.FindBin(0.2),hist.FindBin(0.25))/hist_BG.Integral(hist.FindBin(0.2),hist.FindBin(0.25)))
hist_BG.SetLineColor(2)
hist_BG.Rebin(20)
hist_BG.Scale(1./20)
hist_BG.Draw('same')

integralFG = hist.Integral(hist.FindBin(0.10),hist.FindBin(0.18))
integralBG = hist_BG.Integral(hist_BG.FindBin(0.10),hist_BG.FindBin(0.18))
integral = (integralFG - integralBG)/Nev*1000
print(integral, Nev)

c1.Draw()
c1.SaveAs("output/kek.png")

105.65618192030708 5990000


Info in <TCanvas::Print>: png file output/kek.png has been created


In [24]:
FileOut = ROOT.TFile("input/"+outnames[Option]+".root")
FileOut.ls()

TFile**		output/HeAu_sim_hists_veto_6M_3.root	
 TFile*		output/HeAu_sim_hists_veto_6M_3.root	
  KEY: TH3D	hist_inv_mass_FG8;1	hist_inv_mass_FG8
  KEY: TH3D	hist_inv_mass_BG8;1	hist_inv_mass_BG8
  KEY: TH2D	hist_pt_orig8;1	hist_pt_orig8
